___
- Базовый образ: ubuntu:jammy
- Рабочая директория: /app
- Скопировать локальный файл в контейнер
- Выполняет файл через bash после сборки контейнера

In [ ]:
FROM ubuntu:jammy

WORKDIR /app

COPY script.sh script.sh

ENTRYPOINT ["bash", "script.sh"]

___
- Базовый образ: ubuntu:22.04
- Рабочая директория: /home
- Запустить bash-команду во время сборки контейнера (создает директорию)
- Скопировать локальный файл в созданную директорию контейнера 
- Выполняет файл через bash после сборки контейнера

In [ ]:
FROM ubuntu:22.04

WORKDIR /home

RUN mkdir -p /home/user/app

COPY script.sh /home/user/app/script.sh

CMD ["bash", "/home/user/app/script.sh"]

___
`Запуск телеграм-бота`
- Базовый образ: python:3.8
- Скопировать локальный файл зависимостей в корень контейнера
- Запустить bash-команду во время сборки контейнера (обновление pip, установка зависимостей)
- Рабочая директория: /app
- Скопировать исполняемый файл
- Выполнить исполняемый файл после поднятия контейнера

In [ ]:
FROM python:3.8

COPY requirements.txt requirements.txt

RUN python -m pip install --upgrade pip && pip install -r requirements.txt

WORKDIR /app

COPY tg_bot.py tg_bot.py

ENTRYPOINT [ "python3", "tg_bot.py" ]

___

`Контейнер с логами`

- Базовый образ: python:3.8
- Рабочая директория: /logs
- Скопировать в рабочую директорию контейнера локальный файл:
    - logs_console.py 
    - logs_lib.py
    - logs_file.py
- Выполнить скопированные python-файлы (имя файла указывается при запуске контейнера последним аргументом)
    - -u : вывод будет отображаться сразу же по мере выполнения программы

In [ ]:
FROM python:3.8

WORKDIR /logs

COPY ./logs_console.py ./console

COPY ./logs_lib.py ./lib

COPY ./logs_file.py ./file

ENTRYPOINT ["python3", "-u"]

___

`Облегченный образ для приложения, выводящего hostname контейнера`
- Базовый образ: ubuntu:20.04 из него будут браться только необходимые составляющие
- Рабочая директория: /opt/devops
- Запуск bash-команды:
    - обновление установщика пакетов
    - установка пакетов
    - очистка кэша списка пакетов

- Создание образа с нуля
- Копирование из образа builder необходимых директорий с нужными программами
- Выполнение файла /bin/hostname

In [ ]:
# Первый этап - сборка
FROM ubuntu:20.04 as builder

# Создаём рабочую директорию
WORKDIR /opt/devops

# Устанавливаем необходимые зависимости
RUN apt-get update && apt-get install -y libc6 hostname && rm -rf /var/lib/apt/lists/*

# Второй этап - минимальный образ
FROM scratch

# Копируем необходимые библиотеки для запуска команды hostname
COPY --from=builder /lib64/ld-linux-x86-64.so.2 /lib64/
COPY --from=builder /lib/x86_64-linux-gnu/libc.so.6 /lib/x86_64-linux-gnu/
COPY --from=builder /usr/bin/hostname /usr/bin/sh /bin/

# Устанавливаем команду по умолчанию
CMD ["/bin/hostname"]